# ELIZA Implementation and Analysis

In [1]:
import re
import random
import json
import string

# Pronoun mapping dictionary for transforming user input
PRONOUN_MAP = {
    'I': 'YOU',
    'ME': 'YOU',
    'MY': 'YOUR',
    'AM': 'ARE',
    'MYSELF': 'YOURSELF',
    'MINE': 'YOURS',
    'YOU': 'I',
    'YOUR': 'MY',
    'YOURS': 'MINE',
    'ARE': 'AM',
    'YOURSELF': 'MYSELF'
}

class Eliza:
    def __init__(self, rules):
        self.rules = rules
        self.memory = []
        self.none_ptr = 0 
        self.last_reass = {} 

    def swap_pronouns(self, text):
        """Transform pronouns from first person to second person and vice versa"""
        if not text:
            return ''
        words = text.split()
        swapped = []
        for word in words:
            if word in PRONOUN_MAP:
                swapped.append(PRONOUN_MAP[word])
            else:
                swapped.append(word)
        return ' '.join(swapped)

    def pattern_to_regex(self, pattern):
        """Convert ELIZA pattern to regex for matching"""
        regex_parts = []
        for p in pattern:
            if p == 0:
                regex_parts.append('(.*?)')  # 0 represents wildcard capture
            else:
                regex_parts.append(re.escape(str(p)))
        regex_str = r'^\s*' + r'\s+'.join(regex_parts) + r'\s*$'
        return re.compile(regex_str)

    def extract_parts(self, pattern, match):
        """Extract captured groups from regex match"""
        groups = match.groups()
        group_idx = 0
        parts = []
        for p in pattern:
            if p == 0:
                captured = groups[group_idx].strip()
                parts.append(captured.split() if captured else [])
                group_idx += 1
            else:
                parts.append([p])
        return parts

    def build_response(self, reassembly, parts):
        """Build response from reassembly template and captured parts"""
        response_parts = []
        for item in reassembly:
            if isinstance(item, int):
                if item > 0 and item <= len(parts):
                    part_text = ' '.join(parts[item - 1])  # 1-indexed
                    transformed = self.swap_pronouns(part_text)
                    response_parts.append(transformed)
            else:
                response_parts.append(item)
        response = ' '.join(response_parts)
        # Clean up punctuation spacing
        response = re.sub(r'\s+([?.!,])', r'\1', response)
        return response

    def generate_response(self, user_input):
        """Main method to generate ELIZA's response to user input"""
        # Normalize the input
        user_input = user_input.upper()
        translator = str.maketrans('', '', string.punctuation)
        cleaned = user_input.translate(translator)
        input_words = cleaned.split()

        if not input_words:
            return self.cycle_none()

        # Find candidate keywords
        candidates = []
        for keyword, data in self.rules.items():
            if keyword == 'NONE':
                continue  # Skip NONE for normal search
            kw_upper = keyword.upper()
            precedence = data.get('precedence', 0)
            try:
                index = input_words.index(kw_upper)
                candidates.append((keyword, precedence, index))
            except ValueError:
                pass

        if not candidates:
            return self.use_memory_or_none()

        # Sort by precedence (higher first), then by position (earlier first)
        candidates.sort(key=lambda x: (-x[1], x[2]))

        # Select the highest precedence keyword
        selected_keyword = candidates[0][0]
        selected_data = self.rules[selected_keyword]

        input_text = ' '.join(input_words)

        # Try to match against rules for this keyword
        for rule in selected_data['rules']:
            decomp = rule['decomp']
            regex = self.pattern_to_regex(decomp)
            match = regex.match(input_text)
            if match:
                parts = self.extract_parts(decomp, match)
                reassemblies = rule['reass']
                
                # Avoid repeating the same reassembly
                if len(reassemblies) > 1 and selected_keyword in self.last_reass:
                    avail = [r for i, r in enumerate(reassemblies) if i != self.last_reass[selected_keyword]]
                    reassembly = random.choice(avail)
                else:
                    reassembly = random.choice(reassemblies)
                self.last_reass[selected_keyword] = reassemblies.index(reassembly)
                response = self.build_response(reassembly, parts)

                # Handle memory save if present
                if 'memory' in rule:
                    mem = rule['memory']
                    mem_decomp = mem['decomp']
                    mem_regex = self.pattern_to_regex(mem_decomp)
                    mem_match = mem_regex.match(input_text)
                    if mem_match:
                        mem_parts = self.extract_parts(mem_decomp, mem_match)
                        mem_save = mem['save']
                        mem_reass = random.choice(mem_save)
                        mem_response = self.build_response(mem_reass, mem_parts)
                        self.memory.append(mem_response)

                return response

        return self.use_memory_or_none()

    def use_memory_or_none(self):
        """Use saved memory or fall back to NONE responses"""
        if self.memory:
            return self.memory.pop()
        else:
            return self.cycle_none()

    def cycle_none(self):
        """Cycle through NONE responses to avoid repetition"""
        none_data = self.rules.get('NONE', {})
        if none_data:
            none_reass = none_data['rules'][0]['reass']
            response = ' '.join(none_reass[self.none_ptr])
            self.none_ptr = (self.none_ptr + 1) % len(none_reass)
            return response
        else:
            return "PLEASE GO ON."

def main(script_file='eliza_script.json'):
    """Main function to run ELIZA interactively"""
    try:
        with open(script_file, 'r') as f:
            rules = json.load(f)
    except FileNotFoundError:
        print(f"Error: Script file '{script_file}' not found.")
        return
    except json.JSONDecodeError:
        print("Error: Invalid JSON in script file.")
        return

    eliza = Eliza(rules)

    # Welcome message
    print("HELLO, I'M ELIZA. WHAT'S ON YOUR MIND TODAY? (TYPE 'QUIT' TO EXIT)")

    while True:
        user_input = input("> ").strip()
        if user_input.lower() in ['quit', 'exit', 'goodbye']:
            print("GOODBYE!")
            break
        response = eliza.generate_response(user_input)
        print(response)

if __name__ == "__main__":
    main()

HELLO, I'M ELIZA. WHAT'S ON YOUR MIND TODAY? (TYPE 'QUIT' TO EXIT)
GOODBYE!


---

## 2. Testing

Below are several example conversations with my ELIZA implementation to demonstrate its capabilities and limitations.


In [2]:
# Load ELIZA and run test conversations
import json

# Load the rules
with open('eliza_script.json', 'r') as f:
    rules = json.load(f)

eliza = Eliza(rules)

def run_conversation(messages, title):
    print(f"Conversation {title}")
    for message in messages:
        print(f"User: {message}")
        response = eliza.generate_response(message)
        print(f"ELIZA: {response}")
        print()


In [3]:
# Test Conversation 1: Family and Feelings
# This conversation specifically uses: HELLO, I AM, I FEEL, FAMILY, MOTHER, ALWAYS, I WANT, I NEED, FATHER, BECAUSE
conversation1 = [
    "Hello",
    "I am feeling sad today",
    "I feel like my family doesn't understand me",
    "My mother always criticizes me",
    "I want to be happy",
    "I need someone to talk to",
    "My father never listens to me",
    "Because he is always working",
    "I am tired of this situation",
    "I feel like no one cares about me"
]

run_conversation(conversation1, "1")


Conversation 1
User: Hello
ELIZA: PLEASE GO ON.

User: I am feeling sad today
ELIZA: THAT IS INTERESTING. PLEASE CONTINUE.

User: I feel like my family doesn't understand me
ELIZA: TELL ME MORE ABOUT YOUR FAMILY.

User: My mother always criticizes me
ELIZA: WHAT ELSE COMES TO MIND WHEN YOU THINK OF YOUR MOTHER?

User: I want to be happy
ELIZA: TELL ME MORE.

User: I need someone to talk to
ELIZA: DO YOU FEEL STRONGLY ABOUT DISCUSSING SUCH THINGS?

User: My father never listens to me
ELIZA: WHAT ELSE COMES TO MIND WHEN YOU THINK OF YOUR FATHER?

User: Because he is always working
ELIZA: WHEN?

User: I am tired of this situation
ELIZA: I SEE.

User: I feel like no one cares about me
ELIZA: ARE YOU SAYING NO JUST TO BE NEGATIVE?



In [4]:
# Test Conversation 2: Computer and Technology
# This conversation specifically uses: COMPUTER, NEVER, EVERYONE, I THINK, I AM, ALWAYS, I WANT, I NEED, BECAUSE, YES
conversation2 = [
    "I hate computers",
    "Computers are taking over the world",
    "I never want to use a computer again",
    "Everyone uses computers now",
    "I think computers are evil",
    "I am always frustrated with technology",
    "I want to live without computers",
    "I need to escape from this digital world",
    "Because computers control everything",
    "Yes, I am sure about this"
]

run_conversation(conversation2, "2")


Conversation 2
User: I hate computers
ELIZA: PLEASE GO ON.

User: Computers are taking over the world
ELIZA: THAT IS INTERESTING. PLEASE CONTINUE.

User: I never want to use a computer again
ELIZA: DO COMPUTERS WORRY YOU?

User: Everyone uses computers now
ELIZA: TELL ME MORE.

User: I think computers are evil
ELIZA: DO YOU FEEL STRONGLY ABOUT DISCUSSING SUCH THINGS?

User: I am always frustrated with technology
ELIZA: CAN YOU THINK OF A SPECIFIC EXAMPLE?

User: I want to live without computers
ELIZA: I SEE.

User: I need to escape from this digital world
ELIZA: PLEASE GO ON.

User: Because computers control everything
ELIZA: THAT IS INTERESTING. PLEASE CONTINUE.

User: Yes, I am sure about this
ELIZA: TELL ME MORE.



In [5]:
# Test Conversation 3: General Life and Philosophy
# This conversation specifically uses: I AM, ALWAYS, I THINK, BECAUSE, YES, NO, I FEEL, I WANT, I NEED, NEVER
conversation3 = [
    "I am always worried about the future",
    "I think life is meaningless",
    "Because nothing I do seems to matter",
    "Yes, I am sure about this",
    "No, I don't think things will get better",
    "I feel like giving up",
    "I want to find meaning in life",
    "I need to understand my purpose",
    "I never feel truly happy",
    "I am always searching for answers"
]

run_conversation(conversation3, "3")


Conversation 3
User: I am always worried about the future
ELIZA: REALLY, ALWAYS?

User: I think life is meaningless
ELIZA: DO YOU FEEL STRONGLY ABOUT DISCUSSING SUCH THINGS?

User: Because nothing I do seems to matter
ELIZA: I SEE.

User: Yes, I am sure about this
ELIZA: PLEASE GO ON.

User: No, I don't think things will get better
ELIZA: THAT IS INTERESTING. PLEASE CONTINUE.

User: I feel like giving up
ELIZA: TELL ME MORE.

User: I want to find meaning in life
ELIZA: DO YOU FEEL STRONGLY ABOUT DISCUSSING SUCH THINGS?

User: I need to understand my purpose
ELIZA: I SEE.

User: I never feel truly happy
ELIZA: NEVER?

User: I am always searching for answers
ELIZA: CAN YOU THINK OF A SPECIFIC EXAMPLE?



#  My thoughts on the conversations
## Conversation 1
- Seems very simple, it detects the fact I stated mother and family quite impressive for the short amount of code implemented. Although still very obvious that this chatbot is very simple in its design and not meant to have complex discussions.
## Conversations 2
- It understands the keyword computers and helps keep the user at ease, understanding that they are uneasy about computers.
## Conversation 3:
- Not too good at handling therapy style questions, but still give grammatically acceptable responses that could make sense. Still seems very simple compared to any modern day chatbots even early versions of ChatGPT.

# How does your experience compare to a modern day LLM?
### Answer: It is very obvious that this Eliza bot is extremely simple and not a complex chatbot meant for large and diverse conversations. While I can see how it seemed super advanced given the technology of the 1960's but in todays world Eliza bot is not even in the same conversation as even the simplest chatbots available. Even the early versions of these chatbots are miles better. This stems from the difference in how their logic is structured. Eliza bot is a very simple way of "If user says this, say that" while modern chatbots are trained through rigorous practice and based on probability

# What would you need to do to make this response more human?
### The best way I would assume to make this response more human would be to extend the eliza_script.json. Doing this would greatly improve the diversity of conversations that Eliza can give reasonable response back to.
